In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import os
import requests
import re 

dartIDDF =pd.DataFrame()
dart_temp =pd.DataFrame()

dart_temp = dart_temp.append(
    {
       "pubTime" :"",
        "cat":"",
        "coID":"",
        "content":"",
        "pubDate":"",
        "pubReqDate" :"",
        "rcpNo":"",
        "comName":""},
    ignore_index =True)

temp_pubTime_idx =dart_temp.columns.get_loc("pubTime")
temp_cat_idx =dart_temp.columns.get_loc("cat")
temp_coID_idx =dart_temp.columns.get_loc("coID")
temp_content_idx =dart_temp.columns.get_loc("content")
temp_pubDate_idx =dart_temp.columns.get_loc("pubDate")
temp_pubReqDate_idx =dart_temp.columns.get_loc("pubReqDate")
temp_rcpNo_idx =dart_temp.columns.get_loc("rcpNo")
temp_comName_idx =dart_temp.columns.get_loc("comName")

#2017.07.24~27일까지의 dart 데이터 추출 
for f in range(24, 27) :
    date="2017.07."+ str(f)
    url_part1 ="""http://dart.fss.or.kr/dsac001/mainAll.do?selectDate="""
    urlDate = url_part1 + date
    res = requests.get(urlDate) 
    soup = bs(res.text, "html.parser")  
    totNum=soup.find_all("p")[8].text
    totNum = re.sub(r"\[.*\] \[총", "", totNum)
    totNum=re.sub(r"건\]","",totNum)
    totNum=int(totNum)
    totPage = (totNum//100) +1
    
    for i in range(1, totPage+1) :
        url_part2 = """&series=&mdayCnt=0&currentPage="""+str(i)
        url = url_part1+ date + url_part2   
        res = requests.get(url) 
        soup = bs(res.text, "html.parser")
        
        tempNumCont = len(soup.find_all("tr"))

        for k in range(1,tempNumCont):
            tempinfo = soup.find_all("tr")[k]

            dart_temp.iat[0, temp_pubTime_idx]=tempinfo.find_all("td", class_="cen_txt")[0].text.strip()
            dart_temp.iat[0, temp_cat_idx]=tempinfo.find_all("td")[1].find("img").get("title")
            dart_temp.iat[0, temp_coID_idx]=re.findall(r"[0-9]{8}",tempinfo.find_all("a")[0].get("href"))[0] 
            tempStr = tempinfo.find_all("a")[1].text.strip()
            tempStr=tempStr.replace("\r", "")
            tempStr=tempStr.replace("\n", "")
            tempStr=tempStr.replace("\t", "")
            tempStr=tempStr.replace(" ", "")
            dart_temp.iat[0, temp_content_idx]=tempStr
            dart_temp.iat[0, temp_pubReqDate_idx]=tempinfo.find_all("td", class_="cen_txt")[1].text.strip()
            if(len(tempinfo.find_all("a"))==2) :
                dart_temp.iat[0, temp_rcpNo_idx]=re.findall(r"[0-9]{14}", tempinfo.find_all("a")[1].get("href"))[0]
            else :
                dart_temp.iat[0, temp_rcpNo_idx]=re.findall(r"[0-9]{14}", tempinfo.find_all("a")[2].get("href"))[0]
            dart_temp.iat[0, temp_comName_idx]=tempinfo.find_all("a")[0].text.strip()
            dart_temp.iat[0, temp_pubDate_idx]=soup.find_all("input")[13].get("value")
            
            dartIDDF=dartIDDF.append(dart_temp,ignore_index = True)

            for j in range(0, len(dart_temp.columns)) :
                dart_temp.iat[0,j] =""

dartIDDF

,cat,coID,comName,content,pubDate,pubReqDate,pubTime,rcpNo
0,코스닥시장,00111865,미래SCI,최대주주변경,2017.07.24,2017.07.24,19:07,20170724900550
1,유가증권시장,00134723,페이퍼코리아,기타경영사항(자율공시),2017.07.24,2017.07.24,18:53,20170724800549
2,유가증권시장,00134723,페이퍼코리아,기타경영사항(자율공시),2017.07.24,2017.07.24,18:49,20170724800548
3,코스닥시장,00268251,다산네트웍스,,2017.07.24,2017.07.24,18:03,20170724900529
4,코스닥시장,00861720,이즈미디어,증권발행실적보고서,2017.07.24,2017.07.24,17:59,20170724000355
...,...,...,...,...,...,...,...,...
894,유가증권시장,00131832,SK디스커버리,투자설명서,2017.07.26,2017.07.26,08:10,20170726000002
895,유가증권시장,00149804,대유에이텍,최대주주등소유주식변동신고서,2017.07.26,2017.07.26,07:58,20170726800003
896,코스닥시장,00308559,코디,주주총회소집공고,2017.07.26,2017.07.26,07:08,20170726000001
897,코스닥시장,00308559,코디,[기재정정]주주총회소집결의,2017.07.26,2017.07.26,07:06,20170726900001
